In [198]:
from torch import nn
import torch
import numpy as np
device = "cpu"
torch.set_default_device(device)
torch.set_default_dtype(torch.float64)

In [262]:
s = torch.nn.Sigmoid()

class linear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.w = torch.nn.Parameter(torch.randn((out_features, in_features)))
        
    def forward(self, x):
        y = torch.matmul(self.w, x)
        return y

class activations(nn.Module):
    def __init__(self):
        super().__init__()
        # self.w = torch.nn.Parameter(torch.randn(4))
        
    def forward(self, x):
        y1 = torch.sin(x[0])
        y2 = torch.cos(x[1])
        y3 = x[2] + x[2]
        y4 = x[3]
        #print(y1,y2,y3,y4)
        ystack = torch.vstack((y1,y2,y3,y4))
        return torch.reshape(ystack, (4, x.shape[1]))

class EQL(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear1 = linear(1, 4)
        self.layer1 = activations()
        self.linear2 = linear(4, 4)

    def forward(self, x):
        y = self.linear1(x)
        y = self.layer1(y)
        y = self.linear2(y)
        return torch.sum(y, dim=0).flatten()

In [265]:
points1 = torch.tensor(np.arange(0.,1.1,0.1))
points2 = torch.tensor(np.arange(1.,2.1,0.1))
x = torch.vstack((points1,points2))
x = torch.unsqueeze(points1, dim=0)
y = torch.cos(x)

model = EQL(1, 4)
#model.linear1.w = nn.Parameter(torch.tensor([[0.],[1.],[0.],[0.]]))
#model.linear2.w = nn.Parameter(torch.tensor([[0.,0.,0.,0.],[0.,0.,0.,1.],[0.,0.,0.,0.],[0.,1.,0.,0.]]))
model(x)

tensor([-2.6294, -2.5632, -2.4965, -2.4295, -2.3623, -2.2953, -2.2285, -2.1623,
        -2.0968, -2.0322, -1.9688], grad_fn=<SumBackward1>)

In [266]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
for t in range(5000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 1000 == 0:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 109.2612176087053
1000 2.518885061591168e-07
2000 2.833431388024466e-10
3000 2.8331904121675805e-10
4000 2.8327820007642293e-10


In [267]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.1416],
        [1.0003],
        [0.0542],
        [0.2923]], requires_grad=True), Parameter containing:
tensor([[-0.2600, -0.6425,  0.4487,  1.4607],
        [-1.3118,  0.8833, -0.4224,  0.8378],
        [-0.0582,  1.0985,  1.2326, -1.2641],
        [ 1.3514, -0.3392, -0.4329, -1.2057]], requires_grad=True)]


In [268]:
y

tensor([[1.0000, 0.9950, 0.9801, 0.9553, 0.9211, 0.8776, 0.8253, 0.7648, 0.6967,
         0.6216, 0.5403]])

In [269]:
model(x)

tensor([1.0000, 0.9950, 0.9801, 0.9553, 0.9211, 0.8776, 0.8253, 0.7648, 0.6967,
        0.6216, 0.5403], grad_fn=<SumBackward1>)